In [3]:
import math

from ib_insync import *
util.startLoop()

ib = IB()
ib.client.setConnectOptions('+PACEAPI')
ib.connect('127.0.0.1', 7497, clientId=20)

<IB connected to 127.0.0.1:7497 clientId=20>

In [60]:
import numpy as np
import yahoo_fin.stock_info as si
import requests
import pandas as pd
import datetime as dt
import os
import os.path
from dateutil.relativedelta import relativedelta
from py_vollib.black.greeks.analytical import delta
from Scripts.Utils import trading_utils
import math
from dateutil import parser
import yfinance as yf
from scipy import interpolate
from decimal import Decimal

In [82]:
def FindMinMaxStrike(expiry, atm_volatility, std_to_look_upto, atm_strike):
    time_to_expiry = trading_utils.tte(dt.datetime.now(), parser.parse(expiry))
    variance_till_expiry = atm_volatility * atm_volatility * time_to_expiry
    vol_till_expiry = math.sqrt(variance_till_expiry)
    percentage_to_look_upto = std_to_look_upto * vol_till_expiry
    min_strike = atm_strike*(1 - percentage_to_look_upto*std_to_look_upto)
    max_strike = atm_strike*(1 + percentage_to_look_upto*std_to_look_upto)
    return min_strike, max_strike


def GetContractSpecsListFromYahooFin(symbol, calendar_dte_limit, std_to_look_upto):
    stock = yf.Ticker(symbol)
    expiry_list = stock.options
    contract_spec_list = []
    for expiry in expiry_list:
        expiry_date_time = parser.parse(expiry)
        if (expiry_date_time - dt.datetime.now()).days > calendar_dte_limit:
            continue
        option_chain = stock.option_chain(date = expiry)
        put_df = option_chain.puts
        put_df = put_df[put_df.inTheMoney == False]
        atm_row = put_df.loc[put_df['strike'].idxmax()]
        atm_volatility = atm_row.impliedVolatility
        atm_strike = atm_row.strike
        min_strike, max_strike = FindMinMaxStrike( expiry, atm_volatility, std_to_look_upto, atm_strike)
        for i, row in put_df.iterrows():
            strike = row['strike']
            if min_strike < strike < max_strike:
                contract_spec_dict = {'Symbol' : symbol, "Expiry" : expiry.replace("-",""), 'Strike' : strike, 'Type' : "P", 'Exchange' : "SMART"}
                contract_spec_list.append(contract_spec_dict)
        call_df = option_chain.calls
        call_df = call_df[call_df.inTheMoney == False]

        for i, row in call_df.iterrows():
            strike = row['strike']
            if min_strike < strike < max_strike:
                contract_spec_dict = {'Symbol' : symbol, "Expiry" : expiry.replace("-",""), 'Strike' : strike, 'Type' : "C", 'Exchange' : "SMART"}
                contract_spec_list.append(contract_spec_dict)
    return contract_spec_list

def GetCurrentPrice(contract):
    temp_contract= contract
    ib.qualifyContracts(temp_contract)
    [ticker] = ib.reqTickers(temp_contract)
    ib.sleep(1)
    return ticker.marketPrice()
def GetInputs(inputs_path):
    df = pd.read_csv(inputs_path, header= None)
    df_dict = dict(zip(df[0], df[1]))
    df_dict['symbols'] = df_dict['symbols'].split("|")
    df_dict['anchors_list'] = df_dict['anchors_list'].split("|")
    df_dict['anchors_list'] = [float(a) for a in df_dict['anchors_list']]
    df_dict['dte_limit_for_storing_implied_vols'] = int(df_dict['dte_limit_for_storing_implied_vols'])
    df_dict['std_to_look_upto'] = float(df_dict['std_to_look_upto'])
    return df_dict
def GetAnchorpointVol(list_of_deltas, list_of_vols, list_of_strikes, anchors_list, interpolation_type = 'linear'):
    anchors = anchors_list
    vol_spine = interpolate.interp1d(list_of_deltas, list_of_vols,bounds_error=False, fill_value=-1, kind = interpolation_type)
    strike_spine = interpolate.interp1d(list_of_deltas, list_of_strikes, bounds_error=False, fill_value=-1)
    vols = []
    fake_strikes = []
    for anchor in anchors:
        vols.append(vol_spine(anchor))
        fake_strikes.append(strike_spine(anchor))
    return (vols, fake_strikes)

def GetContractsFromContractSpecList(contract_spec_list):
    contracts = []
    for contract_spec in contract_spec_list:
        contract = Option(contract_spec['Symbol'], contract_spec['Expiry'], contract_spec['Strike'],contract_spec['Type'], contract_spec['Exchange'])
        contracts.append(contract)
    return contracts

def GetTickersFromContracts(contracts):
    contracts = ib.qualifyContracts(*contracts)
    print("Contracts Qualified. Total number of contracts : ", len(contracts))
    tickers = ib.reqTickers(*contracts)
    print("Tickers Downloaded from IB. Sample Ticker : ", tickers[0])
    return tickers

def GenerateDataframeFromTickes(ticker_list):
    #creating raw dataframe from option tickers
    und_price_dict = {}
    row_list_for_df = []

    for ticker in ticker_list:
        if ticker.modelGreeks == None:
            continue
        expiry = dt.datetime.strptime(ticker.contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
        strike = ticker.contract.strike
        kind = ticker.contract.right
        if ticker.modelGreeks.undPrice == None:
            if symbol in  und_price_dict:
                und_price = und_price_dict[symbol]
            else:
                und_price = GetCurrentPrice(Stock(symbol, 'SMART', 'USD'))
                und_price_dict[symbol] = und_price
        else:
            und_price = ticker.modelGreeks.undPrice
            und_price_dict[symbol] = und_price
        bid_price = ticker.bid
        ask_price = ticker.ask
        mid_price = (bid_price+ask_price)/2
        last_price = ticker.last
        volume = ticker.volume
        ib_bid_delta = ticker.bidGreeks.delta
        data_collection_time = dt.datetime.now()
        ticker_dict_temp = {'data_collection_time' : data_collection_time, 'symbol' : symbol , 'expiry' : expiry, 'strike' : strike, 'kind': kind, 'und_price' : und_price,
                       'bid_price': bid_price, 'ask_price' : ask_price, 'mid_price' : mid_price,
                       'last_price' : last_price, 'volume' : volume, 'ib_bid_delta' : ib_bid_delta }
        row_list_for_df.append(ticker_dict_temp)
    df = pd.DataFrame(row_list_for_df)
    df['tte'] = df.apply(lambda x : trading_utils.tte( x.data_collection_time, dt.datetime.combine(x.expiry, dt.datetime.min.time())), axis = 1)
    df['implied_vol_mid_point'] = df.apply(lambda x : trading_utils.bs_implied_vol(x.und_price, x.strike, x.tte, x.kind, x.mid_price), axis= 1)
    df['mid_point_delta'] =  df.apply(lambda x : trading_utils.bs_delta(x.und_price, x.strike, x.tte, x.kind, x.implied_vol_mid_point), axis = 1)
    df['mid_point_call_delta'] = df.apply(lambda x : x.mid_point_delta if x.mid_point_delta > 0 else (1 - x.mid_point_delta*-1), axis = 1)
    return df


def GenerateNeptuneDataframeFromOptionData(single_df, anchors_list):
    neptune_list_for_df = []
    for symbol in single_df.symbol.unique():
        for expiry in single_df.expiry.unique():
            single_df_temp = single_df[(single_df.expiry == expiry)& (single_df.symbol == symbol)]
            if single_df_temp.shape[0] == 0:
                continue
            list_of_deltas = single_df_temp.mid_point_call_delta.tolist()
            list_of_vols = single_df_temp.implied_vol_mid_point.tolist()
            list_of_strikes = single_df_temp.strike.tolist()
            (vols, fake_strikes) = GetAnchorpointVol(list_of_deltas, list_of_vols, list_of_strikes, anchors_list)
            tte = single_df_temp.tte.iloc[0]
            und_price = single_df_temp.und_price.iloc[0]
            temp_dict = {'symbol' :  symbol, 'expiry' : expiry, 'tte' : tte, 'und_price': und_price}
            i = 0
            for anchor in anchors_list:
                key = str(anchor)
                value = vols[i][()]
                temp_dict[key] = value
                key1 = str(anchor)+ '_str'
                value1 = fake_strikes[i][()]
                temp_dict[key1] = value1
                i = i +1
            neptune_list_for_df.append(temp_dict)
    neptune_df = pd.DataFrame(neptune_list_for_df)
    return neptune_df

def AddToImpliedFile(implied_df, symbol, data_path, symbol_kind):
    file_path = data_path + "/" + symbol_kind + "/" + symbol + "/implied.csv"
    file_exist = os.path.exists(file_path)
    if file_exist:
        implied_df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        implied_df.to_csv(file_path, index = False)


def AddImpliedData(symbol, data_path, symbol_kind, dte_limit, anchors_list, std_to_look_upto):
    contract_spec_list = GetContractSpecsListFromYahooFin(symbol, dte_limit, std_to_look_upto)
    contracts = GetContractsFromContractSpecList(contract_spec_list)
    tickers = GetTickersFromContracts(contracts)
    df = GenerateDataframeFromTickes(tickers)
    neptune_df = GenerateNeptuneDataframeFromOptionData(df,anchors_list)
    neptune_df['data_collection_time'] = dt.datetime.now()
    AddToImpliedFile(neptune_df, symbol, data_path, symbol_kind)
    return






In [83]:
inputs_path = "./../../Inputs/historical_data_inputs.csv"
data_path = "./../../HistoricalData"
inputs_dict = GetInputs(inputs_path)
dte_limit = inputs_dict['dte_limit_for_storing_implied_vols']
anchors_list = inputs_dict['anchors_list']
std_to_look_upto = inputs_dict['std_to_look_upto']

In [84]:
for symbol in inputs_dict['symbols']:
    AddImpliedData(symbol, data_path, "stocks", dte_limit, anchors_list, std_to_look_upto)



Contracts Qualified. Total number of contracts :  18
Tickers Downloaded from IB. Sample Ticker :  Ticker(contract=Option(conId=586424353, symbol='AAPL', lastTradeDateOrContractMonth='20220930', strike=133.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  220930P00133000', tradingClass='AAPL'), time=datetime.datetime(2022, 9, 29, 17, 0, 34, 243789, tzinfo=datetime.timezone.utc), bid=0.1, bidSize=12282.0, ask=0.11, askSize=541.0, last=0.11, lastSize=1.0, volume=1756.0, high=0.13, low=0.06, close=0.05, halted=0.0, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.7082269803195991, delta=-0.04414774407122382, optPrice=0.10000000149011612, pvDividend=0.0, gamma=0.01672088192669689, vega=0.007113771446483466, theta=-0.10000000149011612, undPrice=142.0780029296875), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.7222919710320717, delta=-0.047066382999816726, optPrice=0.10999999940395355, pvDividend=0.0, gamma=0.017286759622053276, vega=0.0071150

In [85]:
ib.disconnect()